In [2]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 704.5 kB/s eta 0:00:00m eta 0:00:010:00:13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 KB 811.1 kB/s eta 0:00:00MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 KB 2.3 MB/s eta 0:00:0013.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 2.5 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.2 MB/s eta 0:00:002.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5

In [4]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from collections import Counter

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from imblearn.over_sampling import SMOTE
import numpy as np

# MALO BOLJI REZ, PRECIZNOST PO KLASI

In [ ]:
files = []
folder = "../proba2/direct"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSet/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSet/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

# Build simple NN
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["categorical_accuracy"])

# Train
model.fit(X_train_np, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

# JOS BOLJI, OVERSAMPLING URADJEN PLUS PRECIZNOST PO KLASI KAO METRIKA

In [ ]:
files = []
folder = "../proba2/direct"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSet/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSet/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

smote = SMOTE(random_state=42, k_neighbors = 2)
X_train_smote, y_train_enc_smote = smote.fit_resample(X_train_np, y_train_enc)

# 2️⃣ One-hot encode again (SMOTE returns encoded labels, not one-hot)
y_train_cat_smote = to_categorical(y_train_enc_smote)

model = Sequential()
model.add(Dense(128, input_dim=X_train_smote.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat_smote.shape[1], activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

# 👇 Train with SMOTE-resampled data!
model.fit(X_train_smote, y_train_cat_smote, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

In [ ]:
# rezultati prethodnog:
"""
Classification report:
              precision    recall  f1-score   support

   HCoV-229E       1.00      1.00      1.00       153
   HCoV-HKU1       0.89      1.00      0.94        99
   HCoV-NL63       1.00      0.99      1.00       214
   HCoV-OC43       0.99      0.99      0.99       366
         IBV       1.00      1.00      1.00      3153
    MERS-CoV       1.00      1.00      1.00       395
    SARS-CoV       1.00      1.00      1.00         3
   SARS-CoV2       1.00      1.00      1.00      2133
     bat-CoV       0.89      1.00      0.94        16
  bovine-CoV       0.99      0.99      0.99       378
  canine-CoV       0.96      0.98      0.97       219
 dolphin-CoV       1.00      1.00      1.00         3
  equine-CoV       1.00      1.00      1.00         7
  feline-CoV       0.99      0.99      0.99       536
  ferret-CoV       1.00      1.00      1.00        13
hedgehog-CoV       1.00      1.00      1.00         6
 porcine-CoV       1.00      1.00      1.00       132
  rabbit-CoV       0.86      1.00      0.92         6
     rat-CoV       0.89      1.00      0.94         8
  turkey-CoV       1.00      1.00      1.00        27

    accuracy                           1.00      7867
   macro avg       0.97      1.00      0.98      7867
weighted avg       1.00      1.00      1.00      7867

Accuracy:  0.9956781492309648
"""

In [ ]:
files = []
folder = "../proba2/indirect"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSet/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSet/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

smote = SMOTE(random_state=42, k_neighbors = 2)
X_train_smote, y_train_enc_smote = smote.fit_resample(X_train_np, y_train_enc)

# 2️⃣ One-hot encode again (SMOTE returns encoded labels, not one-hot)
y_train_cat_smote = to_categorical(y_train_enc_smote)

model = Sequential()
model.add(Dense(128, input_dim=X_train_smote.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat_smote.shape[1], activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

# 👇 Train with SMOTE-resampled data!
model.fit(X_train_smote, y_train_cat_smote, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

In [ ]:
# REZ ZA INDIREKTNE:
"""
Classification report:
              precision    recall  f1-score   support

   HCoV-229E       0.99      1.00      1.00       153
   HCoV-HKU1       0.98      1.00      0.99        99
   HCoV-NL63       1.00      1.00      1.00       214
   HCoV-OC43       0.99      0.98      0.99       366
         IBV       1.00      1.00      1.00      3153
    MERS-CoV       1.00      0.99      0.99       395
    SARS-CoV       1.00      1.00      1.00         3
   SARS-CoV2       1.00      1.00      1.00      2133
     bat-CoV       1.00      0.94      0.97        16
  bovine-CoV       0.99      1.00      0.99       378
  canine-CoV       0.98      0.96      0.97       219
 dolphin-CoV       1.00      1.00      1.00         3
  equine-CoV       0.14      1.00      0.25         7
  feline-CoV       0.99      0.93      0.96       536
  ferret-CoV       1.00      0.92      0.96        13
hedgehog-CoV       1.00      1.00      1.00         6
 porcine-CoV       1.00      1.00      1.00       132
  rabbit-CoV       0.86      1.00      0.92         6
     rat-CoV       0.89      1.00      0.94         8
  turkey-CoV       0.96      0.96      0.96        27

    accuracy                           0.99      7867
   macro avg       0.94      0.98      0.94      7867
weighted avg       1.00      0.99      0.99      7867

Accuracy:  0.9912291852040168
"""

In [5]:
files = []
folder = "../proba2/direct"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSet/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSet/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

# Build simple NN
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train_np, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

Number of unique repeats: 798


/home/pc/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-01 08:57:07.139265: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/20


2025-07-01 08:57:07.398446: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 22503600 exceeds 10% of free system memory.


111/111 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8205 - loss: 0.8493 - val_accuracy: 0.9974 - val_loss: 0.0472
Epoch 2/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9754 - loss: 0.1205 - val_accuracy: 0.9974 - val_loss: 0.0339
Epoch 3/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9858 - loss: 0.0743 - val_accuracy: 1.0000 - val_loss: 0.0063
Epoch 4/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9883 - loss: 0.0503 - val_accuracy: 1.0000 - val_loss: 0.0042
Epoch 5/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.9914 - loss: 0.0373 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 6/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9903 - loss: 0.0321 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 7/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9906 - loss: 0.0349 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 8/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9911 - loss: 0.0259 - val_accuracy: 1.000

In [6]:
files = []
folder = "../proba2/indirect"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSet/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSet"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSet/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

# Build simple NN
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train_np, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

Number of unique repeats: 1007


/home/pc/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-01 08:58:57.309534: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28397400 exceeds 10% of free system memory.


Epoch 1/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8485 - loss: 0.9734 - val_accuracy: 0.9974 - val_loss: 0.1068
Epoch 2/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9800 - loss: 0.1389 - val_accuracy: 1.0000 - val_loss: 0.0142
Epoch 3/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9869 - loss: 0.0583 - val_accuracy: 1.0000 - val_loss: 0.0066
Epoch 4/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9892 - loss: 0.0554 - val_accuracy: 1.0000 - val_loss: 0.0041
Epoch 5/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9838 - loss: 0.0576 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 6/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9908 - loss: 0.0398 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 7/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9885 - loss: 0.0380 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 8/20
111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9908 - loss: 0.0384 - val_accu

In [7]:
files = []
folder = "../proba3/DC/"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSetNucl/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSetNucl/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

# Build simple NN
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train_np, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

Number of unique repeats: 4335


/home/pc/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


2025-07-01 09:04:07.604543: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 121796160 exceeds 10% of free system memory.


109/110 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8735 - loss: 0.8358

2025-07-01 09:04:13.705706: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 13559880 exceeds 10% of free system memory.


110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.8749 - loss: 0.8285 - val_accuracy: 1.0000 - val_loss: 0.0069
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.9906 - loss: 0.0526 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.9960 - loss: 0.0230 - val_accuracy: 1.0000 - val_loss: 0.0027
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9969 - loss: 0.0172 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9985 - loss: 0.0097 - val_accuracy: 1.0000 - val_loss: 0.0024
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9984 - loss: 0.0074 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9969 - loss: 0.0103 - val_accuracy: 1.0000 - val_loss: 0.0029
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9987 - loss: 0.0045 - val_accuracy: 1.000

2025-07-01 09:05:31.212537: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 58262400 exceeds 10% of free system memory.


53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Classification report:
              precision    recall  f1-score   support

    MERS-CoV       1.00      1.00      1.00       395
    SARS-CoV       1.00      1.00      1.00         3
     bat-CoV       1.00      0.93      0.97        15
  bovine-CoV       1.00      1.00      1.00       375
  canine-CoV       0.98      0.99      0.98       218
  feline-CoV       0.99      0.99      0.99       536
hedgehog-CoV       1.00      1.00      1.00         6
 porcine-CoV       1.00      1.00      1.00       132

    accuracy                           0.99      1680
   macro avg       1.00      0.99      0.99      1680
weighted avg       0.99      0.99      0.99      1680

Accuracy:  0.9946428571428572


In [8]:
files = []
folder = "../proba3/DC/"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSetNucl/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSetNucl/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

# Build simple NN
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train_np, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

Number of unique repeats: 4335


/home/pc/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.8953 - loss: 0.7545 - val_accuracy: 1.0000 - val_loss: 0.0043
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9896 - loss: 0.1138 - val_accuracy: 0.9974 - val_loss: 0.0381
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.9929 - loss: 0.1119 - val_accuracy: 1.0000 - val_loss: 0.0046
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9956 - loss: 0.0231 - val_accuracy: 0.9949 - val_loss: 0.0060
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9982 - loss: 0.0265 - val_accuracy: 0.9974 - val_loss: 0.0201
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9988 - loss: 0.0074 - val_accuracy: 0.9974 - val_loss: 0.0087
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9974 - loss: 0.0064 - val_accuracy: 0.9974 - val_loss: 0.0059
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9994 - loss: 0.0029 - val_accu

In [9]:
files = []
folder = "../proba3/IC/"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSetNucl/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSetNucl/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

# Build simple NN
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train_np, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

Number of unique repeats: 5163


/home/pc/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.8908 - loss: 0.6853 - val_accuracy: 0.9949 - val_loss: 0.5406
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9863 - loss: 0.3491 - val_accuracy: 0.9949 - val_loss: 0.0752
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9964 - loss: 0.0260 - val_accuracy: 0.9949 - val_loss: 0.3650
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9950 - loss: 0.1313 - val_accuracy: 1.0000 - val_loss: 8.6486e-05
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9992 - loss: 0.0067 - val_accuracy: 0.9974 - val_loss: 0.0117
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9979 - loss: 0.0082 - val_accuracy: 1.0000 - val_loss: 4.0339e-04
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 4.5531e-04
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 1.0000 - loss: 0.001

In [10]:
files = []
folder = "../proba3/IN/"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labels = []
repeats = []

# Read data from each file
for file in files:
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            if len(parts) >= 2:
                label = parts[0]
                repeat = parts[1]
                labels.append(label)
                repeats.append(repeat)

files = []
folder = "../trainSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)

labelTrain = []
sequencesTrain = []

# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTrain.append(record.seq)
        labelTrain.append(file.replace("../trainSetNucl/", "").replace(".fasta", ""))

labelTest = []
sequencesTest = []

files = []
folder = "../testSetNucl"

for file in os.listdir(folder):
    if file.startswith("."):
        continue
    files.append(folder + "/" + file)


# Read data from each file
for file in files:
    records = SeqIO.parse(file, "fasta")
    for record in records:
        sequencesTest.append(record.seq)
        labelTest.append(file.replace("../testSetNucl/", "").replace(".fasta", ""))

known_repeats = list(set(repeats))
print(f"Number of unique repeats: {len(known_repeats)}")

def extract_repeat_counts(sequence, repeat_list):
    return [sequence.count(r) for r in repeat_list]

X_train = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTrain]
y_train = labelTrain

X_test = [extract_repeat_counts(seq, known_repeats) for seq in sequencesTest]
y_test = labelTest

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

# One-hot encode for NN
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Make sure input is NumPy
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

# Build simple NN
model = Sequential()
model.add(Dense(128, input_dim=X_train_np.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # multi-class

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train_np, y_train_cat, epochs=20, batch_size=32, validation_split=0.1)

# Predict
y_pred_probs = model.predict(X_test_np)
y_pred_enc = np.argmax(y_pred_probs, axis=1)
y_pred = encoder.inverse_transform(y_pred_enc)

# Report
print("Classification report:")
print(classification_report(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))

Number of unique repeats: 4897


/home/pc/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9205 - loss: 0.6652 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.9908 - loss: 0.1376 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9964 - loss: 0.0174 - val_accuracy: 0.9974 - val_loss: 0.0087
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9980 - loss: 0.0147 - val_accuracy: 0.9974 - val_loss: 0.0133
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9978 - loss: 0.0162 - val_accuracy: 1.0000 - val_loss: 2.8414e-04
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9991 - loss: 0.0067 - val_accuracy: 1.0000 - val_loss: 2.3215e-04
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9990 - loss: 0.0035 - val_accuracy: 1.0000 - val_loss: 2.5754e-04
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9994 - loss: 0.002